In [0]:
pip install requests requests_oauthlib


In [0]:
pip install pandas


In [0]:
import pandas as pd
from requests_oauthlib import OAuth1Session
import json
import csv

CK = "**********************" #Consumer key
CS = "**********************" #Consumer secret
AT = "**********************" #Access token
ATS = "**********************" #Access token secret

max_id = -1
url = 'https://api.twitter.com/1.1/search/tweets.json'


'''
CSVファイル準備
'''
output_csv_file_name = 'retweet_info.csv' #取得した値を格納ファイル名

with open(output_csv_file_name,'a') as f:
    writer = csv.writer(f, lineterminator='\n')
    header = ['ユーザー名','ユーザー表示名','ユーザーID','リツイートID','時間', 'ツイート']
    writer.writerow(header)
'''
keyword = '' #検索キーワード設定
'''
moto_tweet = '　' #検索ツイート
moto_tweet_screen_name = '@user' #元ユーザー表示名


keyword = moto_tweet +' filter:retweets '+ moto_tweet_screen_name #リツイート検索限定

count = 100
params = {'q' : keyword, 'count' : count, 'max_id' : max_id}

url = "https://api.twitter.com/1.1/search/tweets.json"
twitter = OAuth1Session(CK, CS, AT, ATS)
req = twitter.get(url, params = params)

from time import sleep
 
columns = ['ユーザー名','ユーザー表示名','ユーザーID','リツイートID','時間', 'ツイート']
df = pd.DataFrame(columns=columns)

i = 0 #JSON位置 
while(True):
    if max_id != -1:
        params['max_id']  = max_id - 1  
    req = twitter.get(url, params = params)
    
    if req.status_code == 200:
        search_timeline = json.loads(req.text)
        i += 1 
        
        if search_timeline['statuses'] == []:
            break
        
        for tweet in search_timeline['statuses']:
            user_name = tweet['user']['name']
            user_screen_name = tweet['user']['screen_name']
            user_id =  tweet['user']['id']
            retweet_id = tweet['id']
            created_at = tweet['created_at']
            text = tweet['text']
            
            append_list = [user_name,user_screen_name, user_id, retweet_id, created_at, text]
            
            df_next = pd.DataFrame(
                [append_list],
                columns=columns  
            )

            df = df.append(df_next)
            max_id = search_timeline['statuses'][-1]['id']

            with open(output_csv_file_name, 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow(append_list)

    else: 
        print('1５分待ちます')
        sleep(15*60)

'''
データフレームの確認用
'''
print('dataframeの行数・列数の確認==>\n', df.shape)
print('indexの確認==>\n', df.index)
print('columnの確認==>\n', df.columns)
print('dataframeの各列のデータ型を確認==>\n', df.dtypes) 

df.head()


dataframeの行数・列数の確認==>
 (203, 6)
indexの確認==>
 Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64', length=203)
columnの確認==>
 Index(['ユーザー名', 'ユーザー表示名', 'ユーザーID', 'リツイートID', '時間', 'ツイート'], dtype='object')
dataframeの各列のデータ型を確認==>
 ユーザー名      object
ユーザー表示名    object
ユーザーID     object
リツイートID    object
時間         object
ツイート       object
dtype: object


In [0]:
df.head()

,ユーザー名,ユーザー表示名,ユーザーID,リツイートID,時間,ツイート
0,🍤鈴木覚夜(仮)🍤,suzuki_kaguya,192842156,1219080732154269696,Mon Jan 20 02:14:30 +0000 2020,RT @shikarare084: どうしてこうなった。 https://t.co/CnAw...
0,まご太,dora8kan,771712553642762240,1219079366740627456,Mon Jan 20 02:09:05 +0000 2020,RT @shikarare084: どうしてこうなった。 https://t.co/CnAw...
0,johndoo🔞,John_Doo5,774806743498969088,1219072946733346817,Mon Jan 20 01:43:34 +0000 2020,RT @shikarare084: どうしてこうなった。 https://t.co/CnAw...
0,ぐーすかP太郎@まんどら,Rembrandt_666,157271071,1219072535867740160,Mon Jan 20 01:41:56 +0000 2020,RT @shikarare084: どうしてこうなった。 https://t.co/CnAw...
0,1192 じゅうしまつ (2階から座薬 by 高田純次☆),ggdat,217849065,1219071700827947009,Mon Jan 20 01:38:37 +0000 2020,RT @shikarare084: どうしてこうなった。 https://t.co/CnAw...


データ分析で頻出のPandas
https://qiita.com/ysdyt/items/9ccca82fc5b504e7913a

ソース元
https://datumstudio.jp/blog/twitterapi%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E3%82%AF%E3%83%AD%E3%83%BC%E3%83%A9%E3%83%BC%E4%BD%9C%E6%88%90

【Pandas】データフレームから特定の行・列を取得する方法を総まとめ！
https://yolo.love/pandas/loc-iloc-at-iat/

キーワード項目
https://qiita.com/mida12251141/items/97bce06e4167db8ed3d5

IDのチェック
https://idtwi.com/
